In [1]:
import tensorflow as tf

In [16]:
def convert_and_save_tflite_model(keras_model_path, final_saved_filepath_name):
    try:
        # Basic loading approach
        model = tf.keras.models.load_model(keras_model_path, compile=False)
        
        # Simple conversion without optimizations
        converter = tf.lite.TFLiteConverter.from_keras_model(model)
        tflite_model = converter.convert()
        
        with open(final_saved_filepath_name, 'wb') as f:
            f.write(tflite_model)
        
        print("TFLite model saved at:", final_saved_filepath_name)
    except Exception as e:
        print(f"Error converting model: {str(e)}")

In [17]:
finetuned_stratified_classifmodel = r'C:\Users\benat\OneDrive\0. MSc MLiS\0. GitHub Repositories\PICAR-autopilot\autopilot\models\BenTyler_extradata_v3small\finetunedstratifiedsplit_extrabendata_v3small_speedclassif_model_earlystop.keras'
tflite_finetuned_stratified_classif = r'C:\Users\benat\OneDrive\0. MSc MLiS\0. GitHub Repositories\PICAR-autopilot\autopilot\models\BenTyler_extradata_v3small\tflite_stratsplit_finetuned_extrabendata_v3small_speedclassif_model_earlystop.tflite'

finetuned_classifmodel = r'C:\Users\benat\OneDrive\0. MSc MLiS\0. GitHub Repositories\PICAR-autopilot\autopilot\models\BenTyler_extradata_v3small\finetuned_extrabendata_v3small_speedclassif_model_earlystop.keras'
tflite_finetuned_classifmodel = r'C:\Users\benat\OneDrive\0. MSc MLiS\0. GitHub Repositories\PICAR-autopilot\autopilot\models\BenTyler_extradata_v3small\tflite_finetuned_extrabendata_v3small_speedclassif_model_earlystop.tflite'

finetuned_regressmodel = r'C:\Users\benat\OneDrive\0. MSc MLiS\0. GitHub Repositories\PICAR-autopilot\autopilot\models\BenTyler_extradata_v3small\bendata_regression_finetune_earlystop_mbnetv3small_model.keras'
tflite_finetuned_regressmodel = r'C:\Users\benat\OneDrive\0. MSc MLiS\0. GitHub Repositories\PICAR-autopilot\autopilot\models\BenTyler_extradata_v3small\tflite_bendata_regression_finetune_earlystop_mbnetv3small_model.tflite'

In [18]:
convert_and_save_tflite_model(finetuned_stratified_classifmodel, tflite_finetuned_stratified_classif)
convert_and_save_tflite_model(finetuned_classifmodel, tflite_finetuned_classifmodel)
convert_and_save_tflite_model(finetuned_regressmodel, tflite_finetuned_regressmodel)

Error converting model: Layer 'Conv' expected 1 variables, but received 0 variables during loading. Expected: ['Conv/kernel:0']
Error converting model: Layer 'Conv' expected 1 variables, but received 0 variables during loading. Expected: ['Conv/kernel:0']
Error converting model: Layer 'Conv' expected 1 variables, but received 0 variables during loading. Expected: ['Conv/kernel:0']


Error loading C:\Users\benat\OneDrive\0. MSc MLiS\0. GitHub Repositories\PICAR-autopilot\autopilot\models\BenTyler_extradata_v3small\finetunedstratifiedsplit_extrabendata_v3small_speedclassif_model_earlystop.keras: Layer 'Conv' expected 1 variables, but received 0 variables during loading. Expected: ['Conv/kernel:0']
Trying alternative approaches...
This would require building the model architecture from scratch
and loading weights separately, which needs to be customized per model.
Error loading C:\Users\benat\OneDrive\0. MSc MLiS\0. GitHub Repositories\PICAR-autopilot\autopilot\models\BenTyler_extradata_v3small\finetuned_extrabendata_v3small_speedclassif_model_earlystop.keras: Layer 'Conv' expected 1 variables, but received 0 variables during loading. Expected: ['Conv/kernel:0']
Trying alternative approaches...
This would require building the model architecture from scratch
and loading weights separately, which needs to be customized per model.
Error loading C:\Users\benat\OneDrive\0

In [26]:
def rebuild_model_architecture(input_shape=(224, 224, 3), num_classes=1, dropoutrate=0.2):
    """Rebuild the exact model architecture as provided"""
    # Create base model
    mobnetv3small = tf.keras.applications.MobileNetV3Small(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )
    
    # Create sequential model with exact architecture
    model = tf.keras.Sequential([
        mobnetv3small,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dropout(dropoutrate),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(dropoutrate),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(dropoutrate),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(dropoutrate),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(num_classes, activation='sigmoid')  # Changed to 1 output
    ])
    
    # Set trainable status for mobilenetv3small
    mobnetv3small.trainable = True
    
    return model

In [27]:
import os

def convert_model_with_weights_to_tflite(weights_path, output_path, input_shape=(224, 224, 3), num_classes=1, dropoutrate=0.2):
    """Build model from scratch, load weights, and convert to TFLite"""
    try:
        print(f"Rebuilding model architecture with {num_classes} output classes...")
        model = rebuild_model_architecture(input_shape, num_classes, dropoutrate)
        
        # Load weights (not the full model)
        print(f"Loading weights from {weights_path}...")
        model.load_weights(weights_path)
        print("Weights loaded successfully!")
        
        # Convert to TFLite
        print(f"Converting model to TFLite...")
        converter = tf.lite.TFLiteConverter.from_keras_model(model)
        
        # Enable optimizations
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        
        # Use FP16 precision
        converter.target_spec.supported_types = [tf.float16]
        
        # Convert the model
        tflite_model = converter.convert()
        
        # Ensure directory exists
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        
        # Save the TFLite model
        with open(output_path, 'wb') as f:
            f.write(tflite_model)
        
        print(f"TFLite model saved at: {output_path}")
        model_size_mb = os.path.getsize(output_path) / (1024 * 1024)
        print(f"Model size: {model_size_mb:.2f} MB")
        
        return True
    except Exception as e:
        print(f"Error in conversion: {str(e)}")
        return False


In [32]:
# Define your paths
weights_path = r'C:\Users\benat\OneDrive\0. MSc MLiS\0. GitHub Repositories\PICAR-autopilot\MobNetV3Small_Kaggle\weights\bendata_regression_finetune_earlystop_mbnetv3small_weights.weights.keras'
output_path = r"C:\Users\benat\OneDrive\0. MSc MLiS\0. GitHub Repositories\PICAR-autopilot\autopilot\models\BenTyler_extradata_v3small\tflite_extradata_angleregression_finetuned_mobnetv3small.tflite"

# Convert model
success = convert_model_with_weights_to_tflite(weights_path, output_path)
if success:
    print("Conversion completed successfully!")
else:
    print("Conversion failed.")

Rebuilding model architecture with 1 output classes...
Loading weights from C:\Users\benat\OneDrive\0. MSc MLiS\0. GitHub Repositories\PICAR-autopilot\MobNetV3Small_Kaggle\weights\bendata_regression_finetune_earlystop_mbnetv3small_weights.weights.keras...
Weights loaded successfully!
Converting model to TFLite...
INFO:tensorflow:Assets written to: C:\Users\benat\AppData\Local\Temp\tmpec_3nh40\assets


INFO:tensorflow:Assets written to: C:\Users\benat\AppData\Local\Temp\tmpec_3nh40\assets


TFLite model saved at: C:\Users\benat\OneDrive\0. MSc MLiS\0. GitHub Repositories\PICAR-autopilot\autopilot\models\BenTyler_extradata_v3small\tflite_extradata_angleregression_finetuned_mobnetv3small.tflite
Model size: 2.20 MB
Conversion completed successfully!


In [2]:
import numpy as np

np.arange(0,1.01,0.0625)

array([0.    , 0.0625, 0.125 , 0.1875, 0.25  , 0.3125, 0.375 , 0.4375,
       0.5   , 0.5625, 0.625 , 0.6875, 0.75  , 0.8125, 0.875 , 0.9375,
       1.    ])